In [7]:
import pandas as pd
import numpy as np
import os

os.makedirs('data', exist_ok=True)
N = 2000
np.random.seed(42)

df = pd.DataFrame({
    'patient_id': range(N),
    'age': np.random.randint(18, 90, N),
    'gender': np.random.choice(['Male', 'Female'], N),
    'admission_type': np.random.choice(['Emergency', 'Elective', 'Urgent'], N),
    'length_of_stay': np.random.randint(1, 15, N),
    'previous_admissions': np.random.randint(0, 5, N),
    'lab_glucose': np.random.normal(100, 20, N),
    'lab_hemoglobin': np.random.normal(13, 2, N),
    'cholesterol': np.random.normal(180, 30, N),
    'creatinine': np.random.normal(1.0, 0.3, N),
    'blood_pressure': np.random.normal(120, 15, N),
    'heart_rate': np.random.normal(70, 10, N),
    'temperature': np.random.normal(98.6, 0.7, N)
})

proba = (df['age'] - 18)/72*0.4 + ((df['lab_glucose'] - 80)/80)*0.4
proba = proba.clip(0,1)
df['readmitted'] = np.random.binomial(1, proba)

df.to_csv('data/synthetic_patients.csv', index=False)
print("✅ Synthetic dataset created and saved at 'data/synthetic_patients.csv'")


✅ Synthetic dataset created and saved at 'data/synthetic_patients.csv'


In [8]:
import pandas as pd
import boto3
import sagemaker
from sagemaker import get_execution_role
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib

# Load dataset from S3
s3 = boto3.client('s3')
bucket = 'my-patient-risk-data'
file_key = 'synthetic_patients.csv'

df = pd.read_csv(f's3://{bucket}/{file_key}')
print("✅ Data loaded successfully!")

# Split data
X = df.drop('readmitted', axis=1)
X = pd.get_dummies(X, drop_first=True)
y = df['readmitted']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save model locally
joblib.dump(model, 'model.joblib')

# Upload model to S3
s3_resource = boto3.resource('s3')
s3_resource.Bucket(bucket).upload_file('model.joblib', 'model/model.joblib')
print("✅ Model trained and uploaded to S3!")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:298: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


✅ Data loaded successfully!
✅ Model trained and uploaded to S3!


In [9]:
df.head()

,patient_id,age,gender,admission_type,length_of_stay,previous_admissions,lab_glucose,lab_hemoglobin,cholesterol,creatinine,blood_pressure,heart_rate,temperature,readmitted
0,0,69,Female,Elective,11,2,74.437337,12.328329,207.921719,0.930001,108.827951,67.564681,98.580929,1
1,1,32,Male,Emergency,9,2,105.174203,13.717257,103.212547,1.439261,115.745976,54.017945,97.512792,0
2,2,89,Male,Urgent,2,1,121.029850,12.452365,219.993853,0.590491,133.827104,56.809755,97.572340,1
3,3,78,Female,Urgent,10,1,111.387318,14.390645,136.022021,0.990091,115.878389,69.129190,99.693731,1
4,4,38,Female,Emergency,9,0,86.396612,14.548375,195.620069,1.189704,119.188711,70.727395,97.782732,0


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.ensemble import RandomForestClassifier
import joblib

In [12]:
# 2. Create AGE BUCKET feature .
# ---------------------------------------------------------------
df['age_bucket'] = pd.cut(
    df['age'],
    bins=[0, 30, 50, 70, 120],
    labels=False
)

In [14]:
# 3. Target variable
# ---------------------------------------------------------------
# readmitted already 0/1 in your data
df['readmit_30'] = df['readmitted']

y = df['readmit_30']

In [15]:
# 4. Feature selection
# ---------------------------------------------------------------
numeric_features = [
    'age_bucket',
    'length_of_stay',
    'previous_admissions',
    'lab_glucose',
    'lab_hemoglobin',
    'cholesterol',
    'creatinine',
    'blood_pressure',
    'heart_rate',
    'temperature'
]

categorical_features = [
    'gender',
    'admission_type'
]


In [16]:
# 5. Preprocessing
#    - median imputation for numeric
#    - one-hot encoding for categoricals
from sklearn.impute import SimpleImputer

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

In [17]:
# 6. Full model pipeline

model = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("clf", RandomForestClassifier(
            n_estimators=250,
            max_depth=12,
            random_state=42
        ))
    ]
)

In [18]:
# 7. Train-test split

X = df[numeric_features + categorical_features]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [19]:
# 8. Train model

model.fit(X_train, y_train)

,steps,"[('preprocess', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [20]:
# 9. Evaluate
# ---------------------------------------------------------------
pred_prob = model.predict_proba(X_test)[:, 1]

print("AUC:", roc_auc_score(y_test, pred_prob))
print("\nClassification Report:\n", classification_report(y_test, model.predict(X_test)))


AUC: 0.666549264770626

Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.94      0.82       277
           1       0.57      0.19      0.28       123

    accuracy                           0.71       400
   macro avg       0.65      0.56      0.55       400
weighted avg       0.68      0.71      0.65       400



In [38]:
# 10. Save trained model for SageMaker / Streamlit
# ---------------------------------------------------------------
joblib.dump(model, "readmission_model.joblib")

print("Model saved as readmission_model.joblib")


Model saved as readmission_model.joblib


Features included:
Category	Columns
Demographics	age_bucket, gender
Admission factors	admission_type, length_of_stay, previous_admissions
Labs	lab_glucose, lab_hemoglobin, cholesterol, creatinine
Vitals	blood_pressure, heart_rate, temperature
Preprocessing applied:

Missing numeric values → median

Missing categorical values → most frequent

Categorical → OneHotEncoded

Automatic pipeline → works directly in SageMaker for real-time endpoints

Model:

RandomForestClassifier (good baseline)

Ready for endpoint deployment

In [21]:
import sys
print(sys.executable)


/home/ec2-user/anaconda3/envs/python3/bin/python


In [22]:
import streamlit as st
import pandas as pd
import numpy as np
import joblib

# -----------------------------------------------------------------------
# LOAD MODEL + PREPROCESSOR
# -----------------------------------------------------------------------
@st.cache_resource
def load_model():
    try:
        model = joblib.load("model.pkl")
        preprocessor = joblib.load("preprocessor.pkl")
        return model, preprocessor
    except:
        st.error("⚠ Model files not found! Please upload model.pkl and preprocessor.pkl.")
        return None, None

model, preprocessor = load_model()

# -----------------------------------------------------------------------
# PAGE SETUP
# -----------------------------------------------------------------------
st.set_page_config(
    page_title="Patient Readmission Risk Dashboard",
    layout="wide",
    page_icon="🩺"
)

st.markdown("""
    <h1 style='text-align:center; color:#2C3E50;'>🩺 Patient Readmission Risk Prediction</h1>
    <p style='text-align:center; font-size:18px;'>AI-powered dashboard to estimate 30-day readmission risk.</p>
""", unsafe_allow_html=True)

st.write("---")

# -----------------------------------------------------------------------
# SIDEBAR
# -----------------------------------------------------------------------
st.sidebar.title("📊 Navigation")
page = st.sidebar.radio("Go to", ["🏠 Home", "📝 Predict Risk", "📈 Model Info"])

# -----------------------------------------------------------------------
# HOME PAGE
# -----------------------------------------------------------------------
if page == "🏠 Home":
    st.subheader("Overview")
    st.write("""
    This AI system predicts whether a patient is at risk of being readmitted within 
    30 days after discharge.  
    Built using Machine Learning (Random Forest) and deployed on Streamlit.
    """)
    
    st.write("### What this system provides:")
    st.markdown("""
    - ✔ Real-time patient risk prediction  
    - ✔ Automated preprocessing  
    - ✔ Confidence scoring  
    - ✔ Clean medical dashboard UI  
    - ✔ Works with your trained model  
    """)

    st.image(
        "https://cdn-icons-png.flaticon.com/512/2966/2966486.png",
        width=250
    )

# -----------------------------------------------------------------------
# PREDICTION PAGE
# -----------------------------------------------------------------------
elif page == "📝 Predict Risk":
    st.header("📝 Enter Patient Information")

    col1, col2, col3 = st.columns(3)

    with col1:
        age = st.slider("Age", 18, 100, 45)
        gender = st.selectbox("Gender", ["Male", "Female"])
        admission_type = st.selectbox("Admission Type", ["Emergency", "Elective", "Urgent"])

    with col2:
        length_of_stay = st.number_input("Length of Stay (days)", 1, 30, 4)
        previous_admissions = st.number_input("Previous Admissions", 0, 10, 1)
        lab_glucose = st.number_input("Lab Glucose", 50.0, 300.0, 120.0)

    with col3:
        lab_hemoglobin = st.number_input("Hemoglobin Level", 5.0, 20.0, 13.5)
        cholesterol = st.number_input("Cholesterol", 100, 300, 180)
        creatinine = st.number_input("Creatinine", 0.2, 5.0, 1.0)
        blood_pressure = st.number_input("Blood Pressure", 80, 200, 120)
        heart_rate = st.number_input("Heart Rate", 40, 160, 75)
        temperature = st.number_input("Temperature", 95.0, 105.0, 98.6)

    # Predict Button
    if st.button("🔍 Predict Risk"):
        if model is None:
            st.stop()

        # Prepare input data
        input_data = pd.DataFrame([{
            "age": age,
            "gender": gender,
            "admission_type": admission_type,
            "length_of_stay": length_of_stay,
            "previous_admissions": previous_admissions,
            "lab_glucose": lab_glucose,
            "lab_hemoglobin": lab_hemoglobin,
            "cholesterol": cholesterol,
            "creatinine": creatinine,
            "blood_pressure": blood_pressure,
            "heart_rate": heart_rate,
            "temperature": temperature
        }])

        # Preprocess + Predict
        processed = preprocessor.transform(input_data)
        prob = model.predict_proba(processed)[0][1]
        risk = prob * 100

        # Result
        st.write("---")
        st.subheader("📘 Prediction Result")

        if risk >= 70:
            color = "red"
            status = "HIGH RISK"
        elif risk >= 40:
            color = "orange"
            status = "MEDIUM RISK"
        else:
            color = "green"
            status = "LOW RISK"

        st.markdown(f"""
            <h2 style='color:{color}; text-align:center;'>{status}</h2>
            <h3 style='text-align:center;'>Risk Probability: {risk:.2f}%</h3>
        """, unsafe_allow_html=True)

        st.write("---")

        st.write("### 🧾 Patient Summary")
        st.table(input_data)

# -----------------------------------------------------------------------
# MODEL INFO PAGE
# -----------------------------------------------------------------------
elif page == "📈 Model Info":
    st.header("📈 Model Details")

    st.write("""
    - **Model:** Random Forest Classifier  
    - **Dataset:** Synthetic Patient Records  
    - **Features:** Labs, vitals, demographics  
    """)

    st.code("""
Features used:
- age
- gender
- admission_type
- length_of_stay
- previous_admissions
- lab_glucose
- lab_hemoglobin
- cholesterol
- creatinine
- blood_pressure
- heart_rate
- temperature
    """)

    st.success("Model and preprocessing pipeline are loaded successfully!")



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ ❱   1 import streamlit as st                                                                     │
│     2 import pandas as pd                                                                        │
│     3 import numpy as np                                                                         │
│     4 import joblib                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'streamlit'

In [23]:
st.write("Working directory:", os.getcwd())
st.write("Healthcare files:", os.listdir("/home/ec2-user/SageMaker/healthcare"))


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ ❱ 1 st.write("Working directory:", os.getcwd())                                                  │
│   2 st.write("Healthcare files:", os.listdir("/home/ec2-user/SageMaker/healthcare"))             │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'st' is not defined

In [24]:
import joblib

# Save model
joblib.dump(model, "model.pkl")

# Save preprocessor
joblib.dump(preprocessor, "preprocessor.pkl")


['preprocessor.pkl']

In [25]:
print(preprocessor.feature_names_in_)


['age_bucket' 'length_of_stay' 'previous_admissions' 'lab_glucose'
 'lab_hemoglobin' 'cholesterol' 'creatinine' 'blood_pressure' 'heart_rate'
 'temperature' 'gender' 'admission_type']
